In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [3]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [4]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [6]:
campaña = "Instruvalve"

In [14]:

query_empresacontactoconvenio = (
    "select Id,Campa_a__c,Empresa__r.P_liza__c, Empresa__r.Identificador__c, Paciente__r.IdentificationId__pc,Paciente__r.Name,  N_Documento_Titular__c, Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()


100%|██████████| 1/1 [00:02<00:00,  2.78s/it]


,Id,Campa_a__c,Empresa__r.P_liza__c,Empresa__r.Identificador__c,Paciente__r.IdentificationId__pc,Paciente__r.Name,N_Documento_Titular__c,Relaci_n_Con_Asegurado__c
0,a40RN0000011frlYAA,Instruvalve,-,Instruvalve S.A.,100632713,Marcos Javier Alarcon Andrade,100632713,Titular
1,a40RN0000011frmYAA,Instruvalve,-,Instruvalve S.A.,105317476,Luis Alex Pulgar Riquelme,105317476,Titular
2,a40RN0000011frnYAA,Instruvalve,-,Instruvalve S.A.,105692226,Lucia Veronica San Martin San Maartin,105692226,Titular
3,a40RN0000011froYAA,Instruvalve,-,Instruvalve S.A.,106493316,Alexis Francisco Aravena Cuevas,106493316,Titular
4,a40RN0000011frpYAA,Instruvalve,-,Instruvalve S.A.,115527029,Irma Lorena Faray Barahona,106493316,Carga


In [18]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Campa_a__c": "Campaña",
        "Empresa__r.Identificador__c": "Nombre Empresa",        
        "Empresa__r.P_liza__c":"Poliza",
        "Paciente__r.IdentificationId__pc":"Rut",	
        "Paciente__r.Name": "Nombre",
        "N_Documento_Titular__c": "Rut Titular",
        "Relaci_n_Con_Asegurado__c": "Relación",       
    }
)
empresacontactoconvenio["Llave"] = empresacontactoconvenio["Rut Titular"] + empresacontactoconvenio["Rut"]
empresacontactoconvenio["LlavePoliza"] = empresacontactoconvenio["Poliza"] + empresacontactoconvenio["Rut Titular"] + empresacontactoconvenio["Rut"]

In [20]:
empresacontactoconvenio.to_excel("F:/Empresas SF/reportes/para_eliminar.xlsx", index=False)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Campaña,Poliza,Nombre Empresa,Rut,Nombre,Rut Titular,Relación,Llave,LlavePoliza
0,a40RN0000011frlYAA,Instruvalve,-,Instruvalve S.A.,100632713,Marcos Javier Alarcon Andrade,100632713,Titular,100632713100632713,-100632713100632713
1,a40RN0000011frmYAA,Instruvalve,-,Instruvalve S.A.,105317476,Luis Alex Pulgar Riquelme,105317476,Titular,105317476105317476,-105317476105317476
2,a40RN0000011frnYAA,Instruvalve,-,Instruvalve S.A.,105692226,Lucia Veronica San Martin San Maartin,105692226,Titular,105692226105692226,-105692226105692226
3,a40RN0000011froYAA,Instruvalve,-,Instruvalve S.A.,106493316,Alexis Francisco Aravena Cuevas,106493316,Titular,106493316106493316,-106493316106493316
4,a40RN0000011frpYAA,Instruvalve,-,Instruvalve S.A.,115527029,Irma Lorena Faray Barahona,106493316,Carga,106493316115527029,-106493316115527029
